In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None 
from collections import Counter
import ast

### Map predicted interactions to the phylogenetic tree

#### Load and organize data

##### Organize the data of predicted interactions

In [2]:
#Load the predicted interactions from enrichment analysis
df_predicted = pd.read_csv('C:/Users/Elin/Documents/GitHub/predicting-allostery/datafiles/results/predicted_interactions.txt', header=0, index_col=0)

#Create a dataframe of the top hundred organisms
brenda_top_hundred = pd.read_csv('C:/Users/Elin/Documents/GitHub/predicting-allostery/datafiles/support/top_hundred_organisms.txt', index_col=0, header=0, sep='\t')


In [3]:
#Load the dataframe of the interactions that are mapped to the tree
df_interactions_in_tree = pd.read_csv('C:/Users/Elin/Documents/GitHub/predicting-allostery/datafiles/phylotree/interactions.csv', index_col=0)

#Make a list of the interactions (ChEBI+mode) that are mapped to the tree
interactions_in_tree = list(df_interactions_in_tree['Interaction'])

In [4]:
#Filter the dataframe of predicted interactions for those that are in the tree
df_predicted_tree = df_predicted[df_predicted['interaction'].apply(lambda x: x in interactions_in_tree)]

##### Organize the feature data

In [5]:
#Load the file of features downloaded for every EC number + organism pair that is in the tree
df_features = pd.read_csv('C:/Users/Elin/Documents/GitHub/predicting-allostery/datafiles/features/features_for_ECs_in_tree.txt', index_col=0, header=0)

#When downloading features for the EC number + organism pairs, entries for other organisms are also returned.
#These entries are removed by only keeping those that are from the top hundred organisms
df_features_tree = df_features[df_features['Org'].apply(lambda x: x in set(brenda_top_hundred['Org']))]

In [6]:
#One of the top hundred organisms in the data from BRENDA is "mammal"
#In the phylogenetic tree I created, these are the organisms on the "Mammal" branch:
#(See the file 'create_phylotree.ipynb' for how these were retrieved)
mammals = ['Cavia_porcellus',
 'Rattus_norvegicus',
 'Rattus_sp.',
 'Mus_musculus',
 'Cricetulus_griseus',
 'Mesocricetus_auratus',
 'Oryctolagus_cuniculus',
 'Homo_sapiens',
 'Ovis_aries',
 'Bos_taurus',
 'Sus_scrofa',
 'Equus_caballus',
 'Canis_lupus']


#Replace the underscore with space
mammals_new = []
for mammal in mammals:
    mammals_new.append(mammal.replace('_', ' '))

#Set every organism listed as "Mammalia" to the list of mammals
for ind, row in df_features_tree.iterrows():
    if row['Org'] == 'Mammalia':
        df_features_tree['Org'].at[ind] = mammals_new

#Explode dataframe on organism and drop duplicates
df_features_tree_exploded_org = df_features_tree.explode('Org').drop_duplicates(['EC numbers', 'Org', 'Interpro_ids'])

In [7]:
#Make sure that the InterPro ids are evaluated as strings
df_features_tree_exploded_org['Interpro_ids'] = df_features_tree_exploded_org['Interpro_ids'].apply(ast.literal_eval)

#Explode dataframe on interpro ids to get one each row
df_features_tree_exploded_interpro = df_features_tree_exploded_org.explode('Interpro_ids')

#Remove the "InterPro" part of the InterPro ids
df_features_tree_exploded_interpro['Interpro_ids'] = df_features_tree_exploded_interpro['Interpro_ids'].str.replace('InterPro:', '')

In [8]:
#Remove duplicates and reset dataframe index
df_features_tree_exploded_interpro.drop_duplicates(inplace=True)
df_features_tree_exploded_interpro.reset_index(drop=True, inplace=True)

#### Find organisms for which interactions are predicted

In [9]:
#Make groups of features that are associated with each interaction in the tree, and merge with interactions dataframe to get respective EC number
feature_groups = df_predicted_tree.groupby('interaction').agg({'features':set}).reset_index()
feature_groups_ec = pd.merge(feature_groups, df_interactions_in_tree[['EC', 'Interaction']], left_on='interaction', right_on='Interaction').drop('Interaction', axis=1)

#Make a dictionary with each interaction in the tree as keys and an empty list as values
#This dictionary is to be used for finding the organisms for which these interactions are predicted
predicted_organisms = {key: [] for key in list(feature_groups['interaction'])}

In [10]:
#This block of code iterates through all the features that were downloaded for the EC number and organism combinations,
#and checks whether this feature predicts an interaction. 

#Iterate through every row of features retrieved for the EC number + organism combo
for ind1, row1 in df_features_tree_exploded_interpro.iterrows():
    
    #Iterate through every row of features associated with interactions
    for ind2, row2 in feature_groups_ec.iterrows():
        
        #Check if the feature retrieved for the EC+org combo is associated with the interaction and not already noted 
        if (row1['Interpro_ids'] in row2['features']) and (row1['EC numbers'] == row2['EC']) and (row1['Org'] not in predicted_organisms[row2['interaction']]):
            
            #Add the organism to the list of organisms for which the interaction is predicted
            predicted_organisms[row2['interaction']].append(row1['Org'])   

In [11]:
#Add a column to the interactions dataframe to hold the organisms for which the interaction is predicted
df_interactions_in_tree['Org_predicted'] = ''

In [12]:
#This block of code iterates through the dictionary of interactions and organisms for which they are predicted to be present,
#and adds those organisms to the column of predicted organisms in the interactions dataframe.

for interaction, organisms in predicted_organisms.items():
    for ind, row in df_interactions_in_tree.iterrows():
        if '{}:{}'.format(row['ChEBI'], row['Mode']) == interaction:
            df_interactions_in_tree['Org_predicted'][ind] = organisms

In [13]:
#Create a dataframe of the top hundred organisms, minus mammalia
#Results in 99 organisms
df_itol_binary = brenda_top_hundred
df_itol_binary = df_itol_binary[~df_itol_binary['Org'].isin(['Mammalia'])]

#Create the dataframe of binary values for plotting documented and predicted interactions

#Iterate through every organism in the tree
for ind1, row1 in df_itol_binary.iterrows():
    #Iterate through every interaction we are interested in plotting 
    for ind2, row2 in df_interactions_in_tree.iterrows():
        #Make the interaction label
        interaction = f"{row2['EC']} {row2['Met']} {row2['Mode']}"
        
        #Check if the interaction is documented for the organism in question
        if row1['Org'] in row2['Org']:
            #Set the binary value to 1 if interaction is documented
            df_itol_binary.loc[ind1, interaction] = 1
        #Check if the interaction is predicted for the organism in question
        elif row1['Org'] in row2['Org_predicted']:
            #Set the binary value to 0 if interaction is predicted
            df_itol_binary.loc[ind1, interaction] = 0
        else:
            #Set the binary value to 0 if interaction is not documented or predicted
            df_itol_binary.loc[ind1, interaction] = -1
            

In [14]:
#Replace ' ' in organism name with underscore
df_itol_binary['Org'] = df_itol_binary['Org'].replace(' ', '_', regex=True)

#Copy the binary dataframe to clipboard for creating the file that is uploaded to iTOL
df_itol_binary.to_clipboard(index=False)

In [15]:
#Check number of documented and predicted interactions
#The number of documented interactions is given by the number of 1s --> 214 documented interactions
#The number of predicted interactions is given by the number of 0s --> 230 predicted interactions
df_itol_binary.stack().value_counts()

-1.0                      644
1.0                       214
0.0                       132
Homo_sapiens                1
Hordeum_vulgare             1
                         ... 
Candida_albicans            1
Leishmania_donovani         1
Sinorhizobium_meliloti      1
Vigna_radiata               1
Thermus_thermophilus        1
Length: 102, dtype: int64

In [65]:
df_itol_binary.to_clipboard(index=False)

### Validate predictions

#### Make a file of predicted interactions for validation

In [16]:
#Prepare a dataframe
df_predicted_validation = pd.DataFrame(columns=['Organism', 'EC number', 'Metabolite (ChEBI)', 'Mode'])

In [17]:
#Make sure that the set of organisms in the interactions dataframe is evaluated as a set
df_interactions_in_tree['Org'] = df_interactions_in_tree['Org'].apply(ast.literal_eval)

In [18]:
#Iterate through the interactions dataframe to find predicted interactions
for ind, row in df_interactions_in_tree.iterrows():
    #Set EC, ChEBI and mode variables
    EC = row['EC']
    ChEBI = row['ChEBI']
    Mode = row['Mode']

    #Find the organisms for which interactions were predicted and not documented
    Organisms = set(row['Org_predicted']).difference(row['Org'])
    
    #Add the predicted interactions to the dataframe
    for Organism in Organisms:
        df_predicted_validation = pd.concat([pd.DataFrame([[Organism, EC, ChEBI, Mode]], columns=df_predicted_validation.columns), df_predicted_validation], ignore_index=True)
            

In [19]:
#Load the cleaned up interaction data
df = pd.read_csv('C:/Users/Elin/Documents/GitHub/predicting-allostery/datafiles/interactions/BRENDA_interactions_intracellular.txt', header=0, index_col=0)

In [20]:
#Make a dictionary of all metabolite name synonyms
df_mets_grouped_all_names = df[['Met', 'ChEBI']].drop_duplicates().groupby(['ChEBI'])['Met'].apply(set).reset_index()
mets_dict_all_names = pd.Series(df_mets_grouped_all_names.Met.values, index=df_mets_grouped_all_names.ChEBI).to_dict()

#Map the ChEBI ids to all metabolite synonyms
df_predicted_validation['Metabolite (names)'] = df_predicted_validation['Metabolite (ChEBI)'].map(mets_dict_all_names)

In [21]:
#Make a dictionary of enzyme names for each EC number
df_enz_grouped = df[['Enz', 'EC']].drop_duplicates().groupby(['EC'])['Enz'].apply(set).reset_index()
enz_dict = pd.Series(df_enz_grouped.Enz.values, index=df_enz_grouped.EC).to_dict()

#Map EC number to enzyme name and change to string
df_predicted_validation['Enzyme (name)'] = df_predicted_validation['EC number'].map(enz_dict)
df_predicted_validation['Enzyme (name)'] = df_predicted_validation['Enzyme (name)'].apply(lambda x: ''.join(x))

In [22]:
#Make a dictionary of modes as symbols and words 
modes_dict = {'-': 'Inhibition', '+': 'Activation'}

#Map modes to words
df_predicted_validation['Mode (act/inh)'] = df_predicted_validation['Mode'].map(modes_dict)

In [23]:
#Save file to csv
df_predicted_validation.to_csv('C:/Users/Elin/Documents/GitHub/predicting-allostery/datafiles/results/predicted_interactions_to_be_validated.txt', index=False, sep='\t')

#### Look into validated predictions

In [47]:
pd.set_option('max_colwidth', 800)

#Load file of evidence retrieved for predicted interactions, with journal info
df_evidence = pd.read_csv('C:/Users/Elin/Documents/GitHub/predicting-allostery/datafiles/results/evidence_of_predictions.csv', sep='\t')

In [48]:
#Look at evidence file
df_evidence

,PM_ID,Organism,EC number,Metabolite (ChEBI),Mode,Metabolite (names),Enzyme (name),Mode (act/inh),search,Title,Abstract,Journal,Year,PMC_ID,DOI
0,20707314,Staphylococcus aureus,2.7.1.40,CHEBI:28013,+,"{'fructose diphosphate', 'D-fructose 1,6-diphosphate', 'D-fructose-1,6-diphosphate', 'fructose-1,6-bisphosphate', 'fructose-1,6-diphosphate', 'fructose 1,6-biphosphate', 'D-fructose-1,6-bisphophate', 'D-fructose-1,6-bisphosphate', 'D-fructose 1-6-diphosphate', 'fructose 1,6-bisphosphate', 'D-fructose 1,6 bisphosphate', 'fructose 1,6-diphosphate', 'D-fructose 1,6-bisphosphate'}",pyruvate kinase,Activation,"((S. aureus[ALL]) OR (Staphylococcus aureus[ALL])) AND ((D-fructose-1,6-diphosphate*[ALL]) OR (fructose 1,6-biphosphate*[ALL]) OR (D-fructose-1,6-bisphophate*[ALL]) OR (D-fructose 1,6 bisphosphate*[ALL]) OR (D-fructose-1,6-bisphosphate*[ALL]) OR (fructose-1,6-diphosphate*[ALL]) OR (D-fructose 1,6-diphosphate*[ALL]) OR (fructose 1,6-diphosphate*[ALL]) OR (D-fructose 1-6-diphosphate*[ALL]) OR (fructose 1,6-bisphosphate*[ALL]) OR (fructose diphosphate*[ALL]) OR (D-fructose 1,6-bisphosphate*[ALL]) OR (fructose-1,6-bisphosphate*[ALL])) AND ((pyruvate kinase*[ALL])) AND ((Activat*[ALL])) AND ((alloster*[TIAB]) OR (allozyme*[TIAB])) AND ((effector*[TIAB]) OR (modulator*[TIAB]) OR (regulator*[TIAB]) OR (*enzyme*[TIAB]) OR (isozyme*[TIAB]))","Functional analysis, overexpression, and kinetic characterization of pyruvate kinase from methicillin-resistant Staphylococcus aureus.","Novel antimicrobial targets are urgently needed to overcome rising antibiotic resistance of important human pathogens including methicillin-resistant Staphylococcus aureus (MRSA). Here we report the essentiality and kinetic properties of MRSA pyruvate kinase (PK). Targetron-mediated gene disruption demonstrated PK is essential for S. aureus growth and survival, suggesting that this protein may be a potential drug target. The presence of the pfk (6-phosphofructokinase)-pyk operon in MRSA252, and the nonessential nature of PFK shown by targetron, further emphasized the essential role of PK in cell viability. The importance of PK in bacterial growth was confirmed by showing that its enzymatic activity peaked during the logarithmic phase of S. aureus growth. PK from Staphylococcus and seve...",Biochemistry,2010,None,10.1021/bi100780t
1,30419357,Mycobacterium tuberculosis,2.7.1.40,CHEBI:28013,+,"{'fructose diphosphate', 'D-fructose 1,6-diphosphate', 'D-fructose-1,6-diphosphate', 'fructose-1,6-bisphosphate', 'fructose-1,6-diphosphate', 'fructose 1,6-biphosphate', 'D-fructose-1,6-bisphophate', 'D-fructose-1,6-bisphosphate', 'D-fructose 1-6-diphosphate', 'fructose 1,6-bisphosphate', 'D-fructose 1,6 bisphosphate', 'fructose 1,6-diphosphate', 'D-fructose 1,6-bisphosphate'}",pyruvate kinase,Activation,"((M. tuberculosis[ALL]) OR (Mycobacterium tuberculosis[ALL])) AND ((D-fructose-1,6-diphosphate*[ALL]) OR (fructose 1,6-biphosphate*[ALL]) OR (D-fructose-1,6-bisphophate*[ALL]) OR (D-fructose 1,6 bisphosphate*[ALL]) OR (D-fructose-1,6-bisphosphate*[ALL]) OR (fructose-1,6-diphosphate*[ALL]) OR (D-fructose 1,6-diphosphate*[ALL]) OR (fructose 1,6-diphosphate*[ALL]) OR (D-fructose 1-6-diphosphate*[ALL]) OR (fructose 1,6-bisphosphate*[ALL]) OR (fructose diphosphate*[ALL]) OR (D-fructose 1,6-bisphosphate*[ALL]) OR (fructose-1,6-bisphosphate*[ALL])) AND ((pyruvate kinase*[ALL])) AND ((Activat*[ALL])) AND ((alloster*[TIAB]) OR (allozyme*[TIAB])) AND ((effector*[TIAB]) OR (modulator*[TIAB]) OR (regulator*[TIAB]) OR (*enzyme*[TIAB]) OR (isozyme*[TIAB]))",Allosteric regulation of pyruvate kinase from Mycobacterium tuberculosis by metabolites.,"Mycobacterium tuberculosis (Mtb) causes both acute tuberculosis and latent, symptom-free infection that affects roughly one-third of the world's population. It is a globally important pathogen that poses multiple dangers. Mtb reprograms its metabolism in response to the host niche, and this adaptation contributes to its pathogenicity. Knowledge of th